In [9]:
import os
import sys
from glob import glob
from gensim.sklearn_api import D2VTransformer
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import models
from gensim import corpora
from gensim.utils import simple_preprocess, lemmatize
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from joblib import dump
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

In [24]:
# def clean(corpus):
#     """ Preprocess the corpus and remove unnecessary data """

#     # Get a set of English stopwords.
#     stopwords = set(nltk.corpus.stopwords.words('english'))

#     # Get a set of punctuation characters.
#     punctuation = set(string.punctuation)

#     # Tool for lemmatizing words.
#     lemmatizer = WordNetLemmatizer()

#     # Remove stop words.
#     stopwords_free = " ".join(
#             [word for word in corpus.lower().split() if word not in stopwords])

#     # Remove punctuation.
#     punctuation_free = ''.join(
#             char for char in stopwords_free if char not in punctuation)


#     normalized_corpus = " ".join(
#             lemmatizer.lemmatize(word) for word in punctuation_free.split())

#     return normalized_corpus.split()

In [10]:
#lemmatize
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}

    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)

In [11]:
def clean(essays):
    #text into lowercase tokens
    tokenized_essays = {label: gensim.utils.simple_preprocess(corpus, deacc=True, min_len=2, max_len=15) for (label, corpus) in essays.items()}

    #remove stopwords
    english_stopwords = nltk.corpus.stopwords.words('english')
    tokenized_essays = {label: [w for w in token_lst if w not in english_stopwords] for (label, token_lst) in tokenized_essays.items()}


    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokenized_essays = {label: [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in token_lst if w not in string.punctuation] for (label, token_lst) in tokenized_essays.items()}


In [18]:
root = os.path.dirname(os.path.realpath('__file__'))
essay_path = root + '\\APWA_text\\Essays\\'

files = os.listdir(essay_path)

essays = {}
for file in files:
    # attempt to confidently guess encoding; otherwise, default to ISO-8859-1
    encoding = "ISO-8859-1"
    guess = chardet.detect(open(essay_path + file, "rb").read())
    if (guess["confidence"] >= 0.95):
        encoding = guess["encoding"]
    
    with open(essay_path + file, "r", encoding=encoding) as f:
        essays[file] = f.read()

KeyboardInterrupt: 

In [26]:
# tf_idf = TfidfVectorizer(preprocessor=clean, ngram_range=(1, 2), decode_error='replace', min_df = 3)
# vectorizedEssays = tf_idf.fit_transform(cleanEssays)

model = D2VTransformer(min_count=1, size=5, workers=6)
model = model.fit(cleanEssays)
docvecs = model.transform(cleanEssays)

In [42]:
aggCluster = AgglomerativeClustering(10)

In [43]:
clusters = aggCluster.fit_predict(docvecs)

In [ ]:
# %matplotlib inline

# #set font size of labels on matplotlib plots
# plt.rc('font', size=16)

# #set style of plots
# sns.set_style('white')

# #define a custom palette
# customPalette = ['#630C3A', '#39C8C6', '#D3500C', '#FFB139', '#607F5C', '#1A009E', '#E60000', '#E600BF', '#CE47FF', '#919191', '#000000', '#B37532', '#9EFF3D']
# sns.set_palette(customPalette)
# sns.palplot(customPalette)

In [ ]:
# facet = sns.lmplot(data=clusters, x='x', y='y', hue='label', 
#                    fit_reg=False, legend=True, legend_out=True)

In [44]:
countDict = {x:0 for x in range(10)}
docLists = {x:[] for x in range(10)}
for i in range(len(clusters)):
    countDict[clusters[i]] += 1
    docLists[clusters[i]].append(i) 
    
countDict

{0: 96, 1: 33, 2: 29, 3: 19, 4: 252, 5: 136, 6: 48, 7: 1, 8: 92, 9: 24}

In [45]:
dump(aggCluster, 'hierarchical.joblib')

['hierarchical.joblib']

In [47]:
# freqVecs = [[0] * docvecs[i[0]].getnnz() for i in docLists.values()]

# for cluster, docs in docLists.items():
#     for doc in docs:
#         freqVecs[cluster] = [freqVecs[cluster][i] + vectorizedEssays[doc][i] for i in range(len(freqVecs[cluster]))]
# print(freqVecs)

AttributeError: 'numpy.ndarray' object has no attribute 'getnnz'

In [48]:
fileGroups = []
for _ in range(10):
    fileGroups.append([])
for cluster, docs in docLists.items():
    for doc in docs:
        fileGroups[cluster].append(essays[doc])
        
[len(x) for x in fileGroups]

[96, 33, 29, 19, 252, 136, 48, 1, 92, 24]

In [28]:
open('C:\Users\geoen\OneDrive\Desktop\Code\Senior Project\APWA_text\Essays\essayGroups', 'w', )

array([[ 2.6337068 ,  0.9227169 ,  1.7934034 , -2.9170702 ,  3.600801  ],
       [ 1.53434   ,  0.45210332,  0.8736396 , -1.4088808 ,  3.0523312 ],
       [ 1.3637971 ,  2.2786262 ,  0.53017306, -1.4944946 ,  3.704339  ],
       ...,
       [-0.22130723,  4.800538  ,  3.606748  , -3.344585  ,  5.267728  ],
       [ 0.37406993,  0.555629  ,  0.9422159 , -1.1152442 ,  4.0929728 ],
       [ 0.59261507,  1.6961343 ,  2.730506  , -0.80120766,  3.852472  ]],
      dtype=float32)